The aim of this cleaning notebook is to aggregate raw imported dataframes into cleaned dataframes with the following columns:
- a date column, with the date of the match
- a season column, with the season of the match, under the format 'year1/year2' (e.g. '2012/2013')
- a home and an away column, with the teams involved in the match
- a columns indicating the number of goals scored by the home team, same for the away team
- a categorical column indicating the final result of the match (home for home victory, away for away victory, or draw)
- columns with betting odds for home team victory, away team victory or draws for all bookmakers. Note that for each season, there may be different bookmakers mentionned in the different dataframes

In [ ]:
import os
import sys
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_path)

import pandas as pd

from src.config import load_config
from src.cleaning import import_raw_aggregated_dataset, dataframe_cleaning, convert_dates_column, season

# config.yaml importation
config_file = 'config.yaml'
config_path = os.path.join(root_path, config_file)
config = load_config(config_path)

# Raw data importation

Two dataframes are created:
- a train dataframe, with all seasons excepted the last imported
- a test dataframe, with the last imported season

In [ ]:
raw_data_path = os.path.join(root_path, config['raw_dir'])
raw_df_train = import_raw_aggregated_dataset(config['first_date_first_season'], config['last_date_last_season']-1, raw_data_path)
raw_df_test = import_raw_aggregated_dataset(config['last_date_last_season']-1, config['last_date_last_season'], raw_data_path)

In [ ]:
raw_df_train.head()

# Raw data cleaning

Columns to delete may be different in the two dataframes.

In [ ]:
cols_to_delete_train = ['Div', 'Time', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD',
                        'BbAvD', 'BbMxA', 'BbAvA', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'BbOU', 'BbMx>2.5', 'BbMx<2.5', 'BbAv>2.5',
                        'BbAv<2.5', 'B365>2.5', 'B365<2.5', 'P<2.5', 'P>2.5', 'Max<2.5', 'Max>2.5', 'Avg<2.5', 'Avg>2.5', 'BbAH', 'BbAHh', 'AHh',
                        'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA',
                        'AvgC<2.5', 'AvgC>2.5', 'MaxC<2.5', 'MaxC>2.5', 'AHCh', 'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA',
                        'AvgCAHH', 'AvgCAHA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC<2.5', 'PC>2.5',
                        'PC<2.5', 'BWCA', 'IWCH', 'IWCD', 'IWCA', 'VCCH', 'VCCD', 'VCCA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'PSCH', 'PSCD',
                        'PSCA', 'WHCH', 'WHCD', 'WHCA', 'HTHG', 'HTAG', 'HTR']

cols_to_delete_test = ['Div', 'Time', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD',
                       'AvgA', 'B365>2.5', 'B365<2.5', 'P<2.5', 'P>2.5', 'Max<2.5', 'Max>2.5', 'Avg<2.5', 'Avg>2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH',
                       'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'AvgC<2.5', 'AvgC>2.5', 'MaxC<2.5', 'MaxC>2.5', 'AHCh', 'B365CAHH', 'B365CAHA',
                       'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'B365C>2.5',
                       'B365C<2.5', 'PC<2.5', 'PC>2.5', 'PC<2.5', 'BWCA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'PSCH', 'PSCD', 'PSCA', 'WHCH',
                       'WHCD', 'WHCA', 'HTHG', 'HTAG', 'HTR', 'BFE>2.5', 'BFE<2.5', 'BFEAHH', 'BFEAHA', 'BFEC>2.5', 'BFEC<2.5', 'BFECAHH', 'BFECAHA']

cols_to_rename = {'Date': config['date_column'],
                  'HomeTeam': config['home_column'],
                  'AwayTeam': config['away_column'],
                  'FTHG': config['nb_goals_home_column'],
                  'FTAG': config['nb_goals_away_column'],
                  'FTR': config['final_result_column']}

values_to_rename = {'final_result': {'D': 'draw',
                                     'H': 'home',
                                     'A': 'away'},
                    'home': {'Paris SG': 'PSG',
                             'St Etienne': 'Saint-Etienne',
                             'Evian Thonon Gaillard': 'ETG',
                             'Ajaccio GFCO': 'Gazélec Ajaccio'},
                    'away': {'Paris SG': 'PSG',
                             'St Etienne': 'Saint-Etienne',
                             'Evian Thonon Gaillard': 'ETG',
                             'Ajaccio GFCO': 'Gazélec Ajaccio'}
                   }


cleaned_df_train = dataframe_cleaning(df=raw_df_train,
                                      cols_to_delete=cols_to_delete_train,
                                      cols_to_rename=cols_to_rename,
                                      values_to_rename=values_to_rename)

cleaned_df_test = dataframe_cleaning(df=raw_df_test,
                                     cols_to_delete=cols_to_delete_test,
                                     cols_to_rename=cols_to_rename,
                                     values_to_rename=values_to_rename)

In [ ]:
cleaned_df_train = convert_dates_column(df=cleaned_df_train, date_column=config['date_column'])
cleaned_df_test = convert_dates_column(df=cleaned_df_test, date_column=config['date_column'])

# Addition of the season column
cleaned_df_train.insert(1, 'season', cleaned_df_train.iloc[:, 0].apply(season))
cleaned_df_test.insert(1, 'season', cleaned_df_test.iloc[:, 0].apply(season))

cleaned_df_train.head()

# Missing values management

In [ ]:
# Missing values

print(f'Train dataframe: \n{cleaned_df_train.isna().sum()} \n')
print(f'Test dataframe: \n{cleaned_df_test.isna().sum()}')

Columns with a lot of missing values (> 5% of the lines) will be deleted. Since they are odd variables, columns with less than 5% will be replaced by mean value.

In [ ]:
cols_to_delete_train = ['GBH', 'GBD', 'GBA', 'LBH', 'LBD', 'LBA', 'SJH', 'SJD', 'SJA', 'BSH', 'BSD', 'BSA']
cols_to_delete_test = ['BWH', 'BWD', 'BWA', 'WHH', 'WHD', 'WHA']

cleaned_df_train = dataframe_cleaning(df=cleaned_df_train,
                                      cols_to_delete=cols_to_delete_train)
cleaned_df_test = dataframe_cleaning(df=cleaned_df_test,
                                     cols_to_delete=cols_to_delete_test)

In [ ]:
numeric_cols_train = cleaned_df_train.select_dtypes(include='number').columns
cleaned_df_train[numeric_cols_train] = cleaned_df_train[numeric_cols_train].fillna(cleaned_df_train[numeric_cols_train].mean())

numeric_cols_test = cleaned_df_test.select_dtypes(include='number').columns
cleaned_df_test[numeric_cols_test] = cleaned_df_test[numeric_cols_test].fillna(cleaned_df_test[numeric_cols_test].mean())

In [ ]:
print(f'Train dataframe: \n{cleaned_df_train.isna().sum()} \n')
print(f'Test dataframe: \n{cleaned_df_test.isna().sum()}')

# Cleaned dataframe export

In [ ]:
cleaned_data_path = os.path.join(root_path, config['cleaned_dir'])
df_train_path = os.path.join(cleaned_data_path, f"{config['cleaned_train_df_name']}.csv")
df_test_path = os.path.join(cleaned_data_path, f"{config['cleaned_test_df_name']}.csv")

cleaned_df_train.to_csv(df_train_path, index=False)
cleaned_df_test.to_csv(df_test_path, index=False)